In [1]:
import ROOT
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy.interpolate import interp1d

ROOT.ROOT.EnableImplicitMT(8)

Welcome to JupyROOT 6.22/05


In [2]:
def decay_mother(mother, daugther1_mass, daugther2_mass):

    event  = ROOT.TGenPhaseSpace()
    masses = np.array([daugther1_mass, daugther2_mass], dtype='float64')

    mother = ROOT.TLorentzVector(mother.X(), mother.Y(), mother.Z(), mother.T())
    event.SetDecay(mother, 2, masses)

    weight = event.Generate()
    daugther1  = event.GetDecay(0)
    daugther2  = event.GetDecay(1)

    return mother, daugther1, daugther2

def get_prob(tree_reader, i_event, epsilon, ad_mass, daugther2_mass, br):
    treeReader.SetEntry(i_event)

    mother, ad, gamma = decay_mother(branch, ad_mass, 0)

    decay_length = 80 * br * (1e-5/epsilon)**2 * (ad.E()/1e+3) * (100/(ad_mass*1e+3))**2 # https://arxiv.org/pdf/1708.09389.pdf - page 6

    L1 = 96
    L2 = 116

    prob = ROOT.TMath.Exp(-L1/decay_length)-ROOT.TMath.Exp(-L2/decay_length)

    return mother, ad, prob

ad2ee_pair_br = np.genfromtxt('ad2ee_pair_br.csv', delimiter=',').T
br_e          = interp1d(ad2ee_pair_br[0], ad2ee_pair_br[1], kind='cubic')

In [3]:
ROOT.gSystem.Load("libPhysics.so")
ROOT.gRandom.SetSeed(123)
aD_p4 = ROOT.TLorentzVector(0., 0., 0., 0.)

In [4]:
# set input parameters and scan ranges
det_radius_in  = 0.125
det_radius_out = 0.500
epsilon_range = np.array([1.0e-7, 5.0e-7, 1.0e-6, 5.0e-6, 1.0e-5, 5.0e-5, 1.0e-4, 5.0e-4, 1.0e-3, 5.0e-3, 
                          1.0e-2, 5.0e-2, 1.0e-1])

hadron_mass = 135.0e-3
ad_mass_range = np.linspace(10, 100, len(epsilon_range))
ad_mass_range = 1.0e-3*ad_mass_range
print(epsilon_range, ad_mass_range)

lumi = 300.0e+15 # 300fb^-1
n_generated = 1e+6/1

[1.e-07 5.e-07 1.e-06 5.e-06 1.e-05 5.e-05 1.e-04 5.e-04 1.e-03 5.e-03
 1.e-02 5.e-02 1.e-01] [0.01   0.0175 0.025  0.0325 0.04   0.0475 0.055  0.0625 0.07   0.0775
 0.085  0.0925 0.1   ]


In [5]:
df = pd.DataFrame(columns=['epsilon','mass', 'n_expected'])

In [6]:
path_name        =  "pythia8/pTHat_scan/"
input_file_names = ["pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_1000_1500_crx_7.69E-04_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_0_10_crx_1.17E+09_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_100_200_crx_2.23E+02_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_10_30_crx_2.80E+06_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_1500_2000_crx_3.03E-05_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_2000_-1_crx_1.96E-06_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_200_400_crx_8.65E+00_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_30_50_crx_3.39E+04_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_400_700_crx_2.33E-01_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_50_100_crx_4.51E+03_pb.root",
                    "pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_700_1000_crx_8.06E-03_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_0_10_crx_1.17E+09_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_1000_1500_crx_7.69E-04_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_100_200_crx_2.23E+02_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_10_30_crx_2.80E+06_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_1500_2000_crx_3.03E-05_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_2000_-1_crx_1.96E-06_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_200_400_crx_8.65E+00_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_30_50_crx_3.39E+04_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_400_700_crx_2.33E-01_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_50_100_crx_4.51E+03_pb.root",
                    "pyt8_10MEvt_13TeVpp_Upsilon_4Vec_pTHat_700_1000_crx_8.06E-03_pb.root"]

x_secs = [1.17E+09, 7.69E-04, 2.23E+02, 2.80E+06, 3.03E-05, 1.96E-06, 8.65E+00, 
          3.39E+04, 2.33E-01, 4.51E+03, 8.06E-03, 1.17E+09, 7.69E-04, 2.23E+02, 
          2.80E+06, 3.03E-05, 1.96E-06, 8.65E+00, 3.39E+04, 2.33E-01, 4.51E+03, 8.06E-03]


for ad_mass in ad_mass_range:
    
    for epsilon in epsilon_range:
         
        n_events = 0.0
        
        for input_file_name, pp2hadron_xsec in zip(input_file_names, x_secs):

            print("running for eps=%.2E  ad_mass=%.2E" %(epsilon, ad_mass))
            print("inputfile:%s" %(path_name+input_file_name))
            
            file       = ROOT.TFile.Open(path_name+input_file_name)
            treeReader = ROOT.TTreeReader("tree", file)
            branch     = ROOT.TTreeReaderValue(ROOT.TLorentzVector)(treeReader, "J_Psy_p4")
            n_entries  = treeReader.GetEntries(1)
            
            br_hadron2adgamma = 2 * epsilon**2 * (1-(ad_mass/hadron_mass)**2)**3
            
            for i_event in tqdm(range(int(100))):
                
                mother, ad, prob = get_prob(treeReader, i_event, epsilon, ad_mass, 0.000, br_e(ad_mass))
                
                if ad.Rapidity() > 6.22 and ad.Rapidity() < 7.61:
                    n_events += (lumi) * (pp2hadron_xsec)* (1/n_generated) * br_hadron2adgamma * prob
                    #print("n_expected", n_events)
            
            del treeReader, branch
            
        df = df.append(pd.Series([epsilon, ad_mass, n_events], index=df.columns), ignore_index=True)

df.to_csv('jpsi_prompt2ad.csv')

running for eps=1.00E-07  ad_mass=1.00E-02
inputfile:pythia8/pTHat_scan/pyt8_10MEvt_13TeVpp_J_Psy_4Vec_pTHat_1000_1500_crx_7.69E-04_pb.root


  0%|          | 0/100 [00:00<?, ?it/s]


ReferenceError: attempt to access a null-pointer

Error in <TTreeReaderValue::Get()>: Value reader not properly initialized, did you remember to call TTreeReader.Set(Next)Entry()?
